## Setup

In [19]:
#imports
import os, sys, json, datetime, re, xlrd  # Provides OS-dependent functionality, system-specific parameters, JSON handling, and date/time manipulation
import pandas as pd             # Provides data structures and data analysis tools
from openpyxl import Workbook
import numpy as np              # Supports large, multi-dimensional arrays and matrices
import requests
import glob
import time

from tqdm import tqdm
from functools import reduce
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'
from IPython.display import display_markdown
from cprl_functions.state_capture import thi_states,state_ref, state_coding, state_coding_r, state_pat, state_abv_pat
from cprl_functions.defined_functions import get_recent_file


# Data Setup

Have the following been updated?
- influence score file
- activities score file
- leg_lookup file (key_creation.py)
    - will need to update all_legs_file too
        - created in influence_score.py but powers the leg_lookup file

In [35]:
#Get lookup Data
key_path = r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data\connectors\legislator lookup'
legislators_df = get_recent_file("leg_lookup_*.csv", key_path)
print(legislators_df)
# legislators_df = pd.read_csv(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data\connectors\leg_lookup_df.csv')

#grab scores data
act_dir_path = r'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files\activity scores\2025'
infl_dir_path = r'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files\influence scores\2025'

activity_file = get_recent_file("activity*.csv", act_dir_path)
influence_file = get_recent_file("*infl*.csv", infl_dir_path)


#make dfs
legislators_df = pd.read_csv(legislators_df)
activities_df = pd.read_csv(activity_file)
influence_df = pd.read_csv(influence_file)

print(legislators_df.columns)
legislators_df

C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data\connectors\legislator lookup\leg_lookup_2025_01_31.csv
Index(['full_pk', 'primary_key', 'state_abbreviation', 'first_name',
       'last_name'],
      dtype='object')


,full_pk,primary_key,state_abbreviation,first_name,last_name
0,10100100,101001,AL,Tim,Melson
1,10100200,101002,AL,Tom,Butler
2,10100300,101003,AL,Arthur,Orr
3,10100400,101004,AL,Garlan,Gudger
4,10100500,101005,AL,Greg,Reed
...,...,...,...,...,...
1986,57101500,571015,WV,NaN,NaN
1987,57101601,571016,WV,Jason,Barrett
1988,57101602,571016,WV,Patricia,Rucker
1989,57101701,571017,WV,Eric,Nelson


### Clean up

In [21]:

# #get rid of nan pks
# #rewrite na pks to be numpy nan
# influence_df.loc[influence_df['full_pk'].str.contains('nan'), 'full_pk'] = np.nan

# #drop em
# influence_df = influence_df.dropna(subset=['full_pk']).reset_index(drop = True)


In [36]:

#clean influence
for i,j in enumerate(influence_df['full_pk']):
    if re.search('nan', str(j)):
        print(j)
        influence_df.loc[i,'full_pk'] = np.nan


In [37]:

#forces all pks to int64
all_dfs = [legislators_df,activities_df,influence_df]
for i,df in enumerate(all_dfs):
    # print(*all.columns, sep = ',')
    # print(all.head)
    if df['full_pk'].dtypes == "object":
        df['full_pk'] = pd.to_numeric(df['full_pk'], errors='coerce').astype('Int64')  # Use nullable Int64 if NaNs are present
        all_dfs[i] = df


In [24]:
#narrow down scores dfs
# activities_df = activities_df.loc[:,['full_pk', 'activities_score']]
# influence_df = influence_df.loc[:,['full_pk', 'influence_score']]

# Merge Data (compiled_scores creation)

## First merge

In [32]:
#merging influence and legislators
#merge influence scores
first_merge = pd.merge(legislators_df, influence_df, how="right", on='full_pk', suffixes=('', '_y'))
first_merge.drop(first_merge.filter(regex='_y$').columns, axis=1, inplace=True)

# print(first_merge.to_string())
print(*first_merge.columns, sep=", ")
print(*activities_df.columns, sep=", ")
first_merge


full_pk, primary_key, state_abbreviation, first_name, last_name, party, tenure, influence_score
full_pk, primary_key, first_name, last_name, activities_score, events


,full_pk,primary_key,state_abbreviation,first_name,last_name,party,tenure,influence_score
0,10006300,100063.0,AL,Cynthia,Almond,Republican,4,11.0
1,10006600,100066.0,AL,Alan,Baker,Republican,19,13.0
2,10004900,100049.0,AL,Russell,Bedsole,Republican,5,11.0
3,10008000,100080.0,AL,Chris,Blackshear,Republican,9,12.0
4,10006100,100061.0,AL,Ronald,Bolton,Republican,3,11.0
...,...,...,...,...,...,...,...,...
1980,57100402,571004.0,WV,Eric,Tarr,Republican,13,13.0
1981,57101402,NaN,NaN,NaN,NaN,Republican,3,11.0
1982,57100102,571001.0,WV,Ryan,Weld,Republican,9,17.0
1983,57100502,571005.0,WV,Michael,Woelfel,Democrat,1,NaN


## Second Merge

In [9]:

#merge influence scores
second_merge = pd.merge(first_merge, activities_df, how="left", on='full_pk', suffixes=('', '_y'))
second_merge.drop(second_merge.filter(regex='_y$').columns, axis=1, inplace=True)

# second_merge

## Clean Merge Data

In [10]:

# Function to count non-whitespace events
def count_events(event_str):
    if pd.isna(event_str):  # Check if the value is NaN
        return 0
    # Split by "|" and strip whitespace
    events_list = [event.strip() for event in event_str.split('|') if event.strip()]
    
    return len(events_list)

# Add a new column with the count of events
second_merge['event_count'] = second_merge['events'].apply(count_events)
# print(second_merge.columns)
# print(second_merge.to_string())



In [11]:
#copy dataframe
from datetime import date

final_df = second_merge.copy()
print(*final_df.columns, sep=', ')


full_pk, primary_key, state_abbreviation, first_name, last_name, party, tenure, influence_score, activities_score, events, event_count


In [12]:
#PK Cleanup
for i,j in enumerate(final_df['full_pk']):
    j_pk = re.findall(r'^\d{6}', str(j))[0]
    seat_num = re.findall(r'\d{2}$', str(j))[0]
    # print(f'seat num is {seat_num}')
    # print(f'type is {type(seat_num)}')
    
    if seat_num == '00':
        # print('its a single seat')
        seat_num_v = np.nan
    else:
        seat_num_v = 'Seat ' + seat_num

    
    state_match = re.findall(r'^\d{2}', str(j_pk))
    state = state_coding_r.get(int(state_match[0]))
    chamber = int(re.findall(r'(?<=^\d{2})\d{1}(?=\d{3})', str(j_pk))[0])
    
    if chamber == 0:
        chamber_v = 'House'
    else:
        chamber_v = 'Senate'
    
    district = int(re.findall(r'(?<=^\d{3})\d{3}$', str(j_pk))[0].lstrip('0'))
    district_v = f'District {district}'
    
    
    
    if str(seat_num_v) != 'nan':
        final_df.loc[i,'seat_num'] = seat_num_v
        


    # print('################')
    # print(f'state is {state}')
    # print(f'chamber is {chamber_v}')
    # print(f'district is {district_v}')
    # print(seat_num_v)
    

    final_df.loc[i,'state'] = state
    final_df.loc[i,'chamber'] = chamber_v
    final_df.loc[i,'district'] = district_v

print(*final_df.columns, sep=', ')


final_df.loc[:,['year']] = "2025"
no_tenure_mask = final_df['tenure'] == 1

#clears values for new legislators
final_df.loc[no_tenure_mask, 'activities_score'] = np.nan
final_df.loc[no_tenure_mask, 'events'] = np.nan
final_df.loc[no_tenure_mask, 'event_count'] = np.nan

full_pk, primary_key, state_abbreviation, first_name, last_name, party, tenure, influence_score, activities_score, events, event_count, state, chamber, district, seat_num


## Export

In [13]:
#export
file_name = f'compiled_scores{str(date.today()).replace('-','_')}.xlsx'
csv_file_name = f'compiled_scores{str(date.today()).replace('-','_')}.csv'
final_df.to_excel(fr'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files\compiled scores\2025\{file_name}', index=False)
final_df.to_csv(fr'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files\compiled scores\2025\{csv_file_name}', index=False)
# final_df

# Bills and Legislator Match

## Compiled plus bills
pulls in bills file and adds on the bills sponsored by legislators and the total count

In [14]:
#file set up

#this is from quorum_bill_pulling
#has bills and pk sponsors but no data to accompany it
bills_file = r"C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files\bills\leg_bills_info.xlsx"
bills = pd.read_excel(bills_file)
# print(bills.columns)


In [15]:
#bill and leg merger
#final df comes from compiling data of legislators
#this file has unique rows for legislators and the bills they sponsored along with overall count of bills
compiled_plus_bills = pd.merge(final_df, bills, how="left", on='primary_key')
compiled_plus_bills = compiled_plus_bills.reset_index(drop = True)
compiled_plus_bills = compiled_plus_bills[~(compiled_plus_bills['tenure'] == 1)]



In [16]:
#export
#this exports a file with all of the main info for legislators and the bills they have sponsored
file_name_cb = f'compiled_plus_bills{str(date.today()).replace('-','_')}.xlsx'
csv_file_name_cb = f'compiled_plus_bills{str(date.today()).replace('-','_')}.csv'
compiled_plus_bills.to_excel(fr'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files\compiled scores\2025\{file_name_cb}',sheet_name=f'{file_name_cb.replace(".xlsx", "")}', index=False)
compiled_plus_bills.to_csv(fr'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files\compiled scores\2025\{csv_file_name_cb}', index=False)


## Bills_and_legisltors

In [17]:
#bills and their legislators with the primary focus on bills``
bills_and_legislators = compiled_plus_bills.assign(
    bills=compiled_plus_bills['bill_labels'].str.split('|')  # Split the string into a list
).explode('bills')  # Create a new row for each list element



# Optionally, clean up the brackets
# bills_and_legislators['Values'] = df_expanded['Values'].str.strip('[]')
# print(compiled_plus_bills.columns)
bills_and_legislators['bill_lookup'] = bills_and_legislators['bills'] + "-[" + bills_and_legislators['state'] + "]"

In [ ]:
# bills_file = r'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files\bills\ed_bills_v2.xlsx'

# ed_bills = pd.read_excel(bills_file)

# ed_bills
# bills_and_legislators

In [18]:

#export
file_name_bl = f'bills_and_legislators{str(date.today()).replace('-','_')}.xlsx'
csv_file_name_bl = f'bills_and_legislators{str(date.today()).replace('-','_')}.csv'
bills_and_legislators.to_excel(fr'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files\bills and legislators\2025\{file_name_bl}', sheet_name=f'{file_name_bl.replace(".xlsx", "")}', index=False)
bills_and_legislators.to_csv(fr'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files\bills and legislators\2025\{csv_file_name_bl}', index=False)



In [ ]:
# Generate a list of numbers from 81 to 2834 with a step of 1
numbers = list(range(0, 830))  # 2834 inclusive

for j in numbers:
    print(j)

# Convert to a comma-separated string
comma_separated_numbers = r"/n".join(map(str, numbers))
len(comma_separated_numbers), comma_separated_numbers
